In [49]:
import gurobipy as guro
import numpy as np
import itertools as it

In [8]:
# 1 fourth_moment_max.py
model=guro.Model()
vars={}

#Creating linear program whose fourth moment is as large as possible
def fourth_max(k,m1,m2):
    for i in range(1,k+1):
        vars[i]=model.addVar(lb=0,ub=1,obj=(i**4))
    #Constraint: sum equals 1
    var_sum=guro.LinExpr()
    for k in range(1,k+1):
        var_sum.add(vars[k],1.0)
    model.addConstr(var_sum==1)
    #Constraint: exp_value=specified_value
    exp_sum=guro.LinExpr()
    for k in range(1,k+1):
        exp_sum.add(vars[k],k)
    model.addConstr(exp_sum==m1)
    #Constraint: second_moment=specified_value
    m2_sum=guro.LinExpr()
    for k in range(1,k+1):
        m2_sum.add(vars[k],k**2)
    model.addConstr(m2_sum==m2)
    #Objective: Want the fourth moment to be maxed
    obj=guro.LinExpr()
    for k in range(1,k+1):
        obj.add(vars[k],k**4)
    model.setObjective(obj,guro.GRB.MAXIMIZE)
    #Optimize model
    model.optimize()
    #
    if model.getAttr("Status")==guro.GRB.OPTIMAL:
        for i in range(1,k+1):
            print("Pr(X= "+str(i)+")="+str(vars[i].getAttr("X")))
    else:
        print("Not optimal. Gurobi status code: ",model.getAttr("Status"))

fourth_max(5,3.5,15)
fourth_max(80,10,200)

In [48]:
# 3
#Choosing a set of investments {1,...,7}. Model the following constraints

#Set investments
model=guro.Model()
invest=np.array([1,2,3,4,5,6,7])    #The possible investments
num_invested=np.array([])           #The amount in which we have invested in
#print(invest.shape)
#print(num_invested.shape)

# (a) you cannot invest in all of them
model.addConstr(num_invested.shape<invest.shape)

# (b) You must choose at least one of them
model.addConstr(num_invested.shape>(0,))

# (c) Investment 1 cannot be chosen if investment 3 is chosen
if 3 in num_invested:
    model.addConstr(1 not in num_invested)

# (d) Investment 4 can be chosen only if investment 2 is also chosen
if 2 not in num_invested:
    model.addConstr(4 not in num_invested)

# (e) You must choose either both investments 1 and 5 or neither
if 1 or 5 in num_invested:
    model.addConstr(1 and 5 in num_invested)
else:
    model.addConstr(1 and 5 not in num_invested)

# (f) You must choose either at least one of the investments 1,2,3 or at least two investments from 2,3,5,6
if 1 in num_invested:
    model.addConstr(2 and 3 in num_invested)
elif 2 or 3 in num_invested:
    model.addConstr((1 and 2 and 3 in num_invested) or (2 and 3 and 5 and 6 in num_invested))
elif 5 or 6 in num_invested:
    model.addConstr(2 and 3 and 5 and 6 in num_invested)
else:
    print("Error")

In [61]:
#4 machineschedule.py
#Single machine, several tasks to perform. Can only process one task at a time.
#Goal is to minimize the average weighted completion time of the tasks.

model=guro.Model()

def machine(n_tasks,time,weight):
    model.ModelSense=guro.GRB.MINIMIZE
    
    #start_vars[i] stores x[i]
    start_vars=[]
    for i in range(0,n_tasks):
        start_vars.append(model.addVar(vtype=guro.GRB.CONTINUOUS))

    #order_vars[i,j] stores y_{ij}
    order_vars={}
    for i in range(0,n_tasks):
        for j in range(0,n_tasks):
            if i!=j:
                order_vars[i,j]=model.addVar(vtype=guro.GRB.BINARY)

    #complete_order_vars[k,i,j] will store z_k^{ij}
    complete_order_vars={}
    for i in range(0,n_tasks):
        for j in range(0,n_tasks):
            if i!=j:
                complete_order_vars[i,i,j]=model.addVar(vtype=guro.GRB.CONTINUOUS,lb=0.0)
                complete_order_vars[i,j,i]=model.addVar(vtype=guro.GRB.CONTINUOUS,lb=0.0)

    #Objective
    total_weight=sum(weight)
    obj=guro.LinExpr()
    for i in range(0,n_tasks):
        obj.add(start_vars[i]+time[i],weight[i])
    model.setObjective(obj/total_weight)

    #Constraint
    for i in range(0,n_tasks):
        for j in range(i+1,n_tasks):
            model.addConstr(order_vars[i,j]+order_vars[j,i]==1)

    #Constraint
    total_time=sum(time)
    for i in range(0,n_tasks):
        for j in range(0,n_tasks):
            if i!=j:
                print(i,i,j,print((i,i,j) in complete_order_vars))
                model.addConstr(complete_order_vars[i,i,j]<=total_time*order_vars[i,j])
                model.addConstr(complete_order_vars[i,j,i]<=total_time*order_vars[j,i])
                model.addConstr(complete_order_vars[j,i,j]>=complete_order_vars[i,i,j]+
                                time[i]*order_vars[i,j])
                model.addConstr(start_vars[i]==complete_order_vars[i,i,j]+complete_order_vars[i,j,i])

    #Optimize and print
    model.optimize()
    if model.Status==guro.GRB.OPTIMAL:
        print("Objective: ", model.ObjVal)
        for i in range(0,n_tasks):
            print("Start time "+str(i)+": ",start_vars[i].X)
    else:
        print("Failed to find optimal value")

machine(7,[5,10,17,8,8,14,28],[0.1,0.2,0.3,0.15,0.05,0.05,0.15])

True
0 0 1 None
True
0 0 2 None
True
0 0 3 None
True
0 0 4 None
True
0 0 5 None
True
0 0 6 None
True
1 1 0 None
True
1 1 2 None
True
1 1 3 None
True
1 1 4 None
True
1 1 5 None
True
1 1 6 None
True
2 2 0 None
True
2 2 1 None
True
2 2 3 None
True
2 2 4 None
True
2 2 5 None
True
2 2 6 None
True
3 3 0 None
True
3 3 1 None
True
3 3 2 None
True
3 3 4 None
True
3 3 5 None
True
3 3 6 None
True
4 4 0 None
True
4 4 1 None
True
4 4 2 None
True
4 4 3 None
True
4 4 5 None
True
4 4 6 None
True
5 5 0 None
True
5 5 1 None
True
5 5 2 None
True
5 5 3 None
True
5 5 4 None
True
5 5 6 None
True
6 6 0 None
True
6 6 1 None
True
6 6 2 None
True
6 6 3 None
True
6 6 4 None
True
6 6 5 None
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 2600 Six-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 189 rows, 133 columns and 462 nonzeros
Model fingerprint: 0xd5509a9f
Variable types: 91 c